In [ ]:
#!pip install langchain
#!pip install langchain_openai
#!pip install langchain_community
#!pip install langchain_core

from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from typing import List, TypedDict, Dict


from langchain_community.utils.openai_functions import (
    convert_pydantic_to_openai_function,
)

from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser


import pandas as pd

In [ ]:
class Itinerary(BaseModel):
    day: List[str] = Field(description="a list that indicates the day of the corresponding activity")
    activity_type: List[str] = Field(description="a list of the activities type of the itinerary like breakfast, dinner etc")
    location: List[str] = Field(description="a list of the location that existis in google places of the corresponding activity type")
    description: List[str] = Field(description="a brief description of the activity. The description must be 2-3 word phrases")

# Set up a parser + inject instructions into the prompt template.
# parser = PydanticOutputParser(pydantic_object=Itinerary)

openai_functions = [convert_pydantic_to_openai_function(Itinerary)]

parser = JsonOutputFunctionsParser()

In [ ]:
openai_functions

[{'name': 'Itinerary',
  'description': '',
  'parameters': {'type': 'object',
   'properties': {'day': {'description': 'a list that indicates the day of the corresponding activity',
     'type': 'array',
     'items': {'type': 'string'}},
    'activity_type': {'description': 'a list of the activities type of the itinerary like breakfast, dinner etc',
     'type': 'array',
     'items': {'type': 'string'}},
    'location': {'description': 'a list of the location that existis in google places of the corresponding activity type',
     'type': 'array',
     'items': {'type': 'string'}},
    'description': {'description': 'a brief description of the activity. The description must be 2-3 word phrases',
     'type': 'array',
     'items': {'type': 'string'}}},
   'required': ['day', 'activity_type', 'location', 'description']}}]

In [ ]:

prompt = PromptTemplate(
    template="""Given the user's input below:

    Location: {location}
    Number of People: {n}
    Date: {date}
    Duration (Days): {duration}
    Tell Us About Your Group: {group}
    Briefly describe the trip you envision including the groups interests in activities, food etc.: {description}

    Create an itinerary providing the activities for breakfast, morning activity, lunch, afternoon activity, dinner and evening activity, the location which must exist in google places and a brief description of the activity or the place

    Make sure to include all the itinerary days

    """

    ,
    input_variables=["location", "n", "date", "duration", "group", "description"],

)

In [ ]:
#!pip install openai

import os
import openai
from google.colab import userdata

# Retrieve the OpenAI API key from Colab secrets
openai_key = userdata.get('Openai')

if openai_key:
    os.environ['OPENAI_API_KEY'] = openai_key
else:
    raise ValueError("Openai API key not found. Please set the 'Openai' secret in Colab.")

# Select GPT model
#model = ChatOpenAI(temperature=0, model = 'gpt-4-turbo-preview')
#model = ChatOpenAI(temperature=0, model = 'gpt-3.5-turbo')
model = ChatOpenAI(temperature=0, model='gpt-3.5-turbo-0125')
#model = ChatOpenAI(temperature=0, model='gpt-4')

# Set up the processing chain
chain = prompt | model.bind(functions=openai_functions) | parser


In [ ]:
eg1 = {
    "location": "South Beach Miami, FL",
    "n": "6",
    "date": "November 2026",
    "duration": "3",
    "group": "bachelorette party",
    "description": "boutique hotel with onsite spa, beach clubs with DJs during the day, and high end nightclubs. List at least six restaurant options for dinner That include at least one nice steakhouse and one nice sushi restaurant"}

In [ ]:
output = chain.invoke({"location": eg1["location"], "n": eg1["n"],
              "date": eg1["date"],  "duration": eg1["duration"],
              "group": eg1["group"],
              "description": eg1["description"]})

# return the data frame

df = pd.DataFrame(output)

df

,day,activity_type,location,description
0,Day 1,Breakfast,The Local House,Cozy breakfast spot
1,Day 1,Morning Activity,South Beach,Enjoy the beach
2,Day 1,Lunch,Yardbird Southern Table & Bar,Southern comfort food
3,Day 1,Afternoon Activity,Nikki Beach Miami,Beach club with DJs
4,Day 1,Dinner,Prime 112,Upscale steakhouse
5,Day 1,Evening Activity,LIV Nightclub,High-end nightclub
6,Day 2,Breakfast,The Front Porch Cafe,Casual cafe with outdoor seating
7,Day 2,Morning Activity,South Pointe Park,Park with beautiful views
8,Day 2,Lunch,Joe's Stone Crab,Seafood dining
9,Day 2,Afternoon Activity,Wynwood Walls,Art murals and graffiti


In [ ]:
def calculate_tokens(data):
    # Calculate input tokens
    input_tokens = sum(len(str(value).split()) for value in data.values())

    # Calculate output tokens
    output_tokens = sum(len(str(value).split()) for value in output.values())

    # Calculate total tokens
    total_tokens = input_tokens + output_tokens

    return input_tokens, output_tokens, total_tokens

# Example usage
input_data = eg1

input_tokens, output_tokens, total_tokens = calculate_tokens(input_data)
print(f"Input Tokens: {input_tokens}")
print(f"Output Tokens: {output_tokens}")
print(f"Total Tokens: {total_tokens}")



Input Tokens: 46
Output Tokens: 162
Total Tokens: 208
